In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report


from sklearn.ensemble import GradientBoostingClassifier

# Load data
train_df = pd.read_csv("data/processed/train_processed.csv")
val_df = pd.read_csv("data/processed/validation_processed.csv")

# Split into features and target
X_train = train_df.drop("Attrition", axis=1)
y_train = train_df["Attrition"]

X_val = val_df.drop("Attrition", axis=1)
y_val = val_df["Attrition"]

# Initialize and train Gradient Boosting model
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
gb_model.fit(X_train, y_train)

# Predict and evaluate
gb_predictions = gb_model.predict(X_val)
gb_accuracy = accuracy_score(y_val, gb_predictions)
gb_report = classification_report(y_val, gb_predictions)

print(f"Gradient Boosting Accuracy: {100*gb_accuracy:.2f}%")
print("Gradient Boosting Classification Report:\n", gb_report)

Gradient Boosting Accuracy: 75.83%
Gradient Boosting Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.77      0.77      6124
           1       0.75      0.74      0.75      5619

    accuracy                           0.76     11743
   macro avg       0.76      0.76      0.76     11743
weighted avg       0.76      0.76      0.76     11743

